# Frequent Itemset Mining

### Student: Rodolfo Lerma

Learning Objectives:

- Extract frequent patterns given a corpus of data.
- Find the rules which are interesting and non-obvious for a given domain.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#### First dataframe

In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

The first dataframe contains 541,909 rows with 8 columns.

In [4]:
columns = df.columns.to_list()

#### Second dataframe

In [5]:
df2 = pd.read_excel('online_retail_II.xlsx')
df2.rename({'Invoice':'InvoiceNo', }, axis = 1, inplace = True) #Renaming the columns to match previous dataframe
df2.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [6]:
df3 = df2.set_axis(columns, axis=1, inplace=False)

In [7]:
df3.shape

(525461, 8)

The second dataframe contains 525,461 rows with 8 columns.

### Question 1.1: Concatinate both dataframes to create a single dataframe. Remove any rows where InvoiceNo is Null and Quantity is Negative

In [8]:
frames = [df, df3]
data = pd.concat(frames, axis = 0, ignore_index = True ,sort=False)
data.tail(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1067368,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
1067369,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom


Checking that we do not have duplicate indexes from the 2 dataframes that we just joined.

In [9]:
data.shape

(1067370, 8)

It is possible to see that the number of rows and columns match what we had before in the previous 2 datasets.
Total number of rows: **1,067,370**

In [10]:
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [11]:
data.isnull().sum() #Looking for NULL Values

InvoiceNo           0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     243007
Country             0
dtype: int64

As we can see here there are no `NULL` values for the **InvoiceNo** column.

In [12]:
data = data[data['Quantity'] > 0] #Removing any negative quantity
data.shape #Check point to verify the size of the filtered dataset

(1044420, 8)

In [13]:
data['Quantity'].value_counts()

1       294345
2       159960
12      121745
6        85299
3        72638
         ...  
273          1
274          1
281          1
284          1
1020         1
Name: Quantity, Length: 564, dtype: int64

No `Quantity` value less or equal to zero.

### Question 1.2: Filter the data by only transactions that happened in United Kingdom 

In [14]:
data = data[data['Country'] == 'United Kingdom'] 
data.shape

(961224, 8)

It is possible to see a small reduction in the size of the dataframe.

### Question 1.3: What are the most popular 5 items?

In [15]:
data = data.astype({'StockCode': str}) #Ensuring the same type for all variables in the StockCode column

#Getting the most popular items
items_list = data['StockCode'].unique().tolist()
total_sum_per_item = []
description_summary = []
invoice_summary = []
for i in items_list:
    x = data[data['StockCode'] == i]
    y = x['Quantity'].sum()
    z = x['Description'].iloc[0]
    w  = x['InvoiceNo'].iloc[0]
    total_sum_per_item.append(y)
    description_summary.append(z)
    invoice_summary.append(w)

Alternatively, I could have used `group_by` to obtain the total quantity for each of the items in the dataframe which is a much faster method.

In [16]:
#Getting a dataframe with the summary information on the most popular items
data_summary = pd.DataFrame(list(zip(invoice_summary, items_list, total_sum_per_item, description_summary)),
               columns =['InvoiceNo','StockCode', 'Total_Qty', 'Description'])

In [17]:
#Getting a sorted dataframe
data_sum_sorted = data_summary.sort_values(by = 'Total_Qty', ascending = False)
data_sum_sorted.head()

,InvoiceNo,StockCode,Total_Qty,Description
1341,536615,84077,101464,WORLD WAR 2 GLIDERS ASSTD DESIGNS
0,536365,85123A,92476,WHITE HANGING HEART T-LIGHT HOLDER
121,536386,85099B,89143,JUMBO BAG RED RETROSPOT
142,536390,22197,84149,SMALL POPCORN HOLDER
3935,581483,23843,80995,"PAPER CRAFT , LITTLE BIRDIE"


**We can see the 5 more popular items on this store:**
- World War 2 Gliders
- White Hanging Heart T-Light Holder
- Jumbo Bag Red Retropot
- Small Popcorn Holder
- Paper Craft

### Question 1.4: Filter down the data to include transaction that contain the top 20 items

In [18]:
#Top 20 items
data_sum_sorted.head(20)

,InvoiceNo,StockCode,Total_Qty,Description
1341,536615,84077,101464,WORLD WAR 2 GLIDERS ASSTD DESIGNS
0,536365,85123A,92476,WHITE HANGING HEART T-LIGHT HOLDER
121,536386,85099B,89143,JUMBO BAG RED RETROSPOT
142,536390,22197,84149,SMALL POPCORN HOLDER
3935,581483,23843,80995,"PAPER CRAFT , LITTLE BIRDIE"
2913,541431,23166,77036,MEDIUM CERAMIC TOP STORAGE JAR
9,536367,84879,76021,ASSORTED COLOUR BIRD ORNAMENT
49,536378,21212,72389,PACK OF 72 RETROSPOT CAKE CASES
929,536544,17003,71000,BROCADE RING PURSE
51,536378,21977,46471,PACK OF 60 PINK PAISLEY CAKE CASES


In [19]:
#Code to filter the original dataframe to one containing the 20 most relevant items
first_20_list = data_sum_sorted['StockCode'].head(20).to_list() #Getting a list with the StockCode of the 20 most popular items
data1 = data.copy()
first_20 = data1['StockCode'].isin(first_20_list) #Boolean list if the item is present in the dataframe
index = data1.index 
indexes = index[first_20] #Getting the indexes of the dataframe where these 20 items are present
indices_list = indexes.tolist() #Getting those indexes to a list
print('Example of the first 10 Indexes where the one of the 20 Items is present')
indices_list[:10]

Example of the first 10 Indexes where the one of the 20 Items is present


[0, 9, 46, 49, 66, 86, 96, 98, 99, 122]

In [20]:
#Data Frame where the StockCode is present for the 20 most popular items
data_20 = data1.loc[indices_list] 
data_20.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom
46,536371,22086,PAPER CHAIN KIT 50'S CHRISTMAS,80,2010-12-01 09:00:00,2.55,13748.0,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom
86,536378,22386,JUMBO BAG PINK POLKADOT,10,2010-12-01 09:37:00,1.95,14688.0,United Kingdom
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00,0.42,14688.0,United Kingdom
98,536378,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24,2010-12-01 09:37:00,0.55,14688.0,United Kingdom
99,536378,84991,60 TEATIME FAIRY CAKE CASES,24,2010-12-01 09:37:00,0.55,14688.0,United Kingdom
122,536381,22086,PAPER CHAIN KIT 50'S CHRISTMAS,4,2010-12-01 09:41:00,2.95,15311.0,United Kingdom


The dataframe above only has items related to the 20 most popular list. But we are more interested in the `invoice` that contains those items and many others as well.

In [21]:
#Invoices where at least one of the 20 more relevant items is present
invoice_20 = data_20['InvoiceNo'].unique().tolist() #A list with the invoice number that has a list one of the 20 most popular items
data2 = data.copy()
#Code to filter the original dataframe to one containing the invoice related to purchases of the 20 most popular items 
first_20_invoice = data2['InvoiceNo'].isin(invoice_20) #Getting a dataframe where the 
index_invoice = data2.index
indexes_invoice = index_invoice[first_20_invoice]
indices_list_invoice = indexes_invoice.tolist() #List of indexes where the invoice is present

In [22]:
# Data Frame with invoice includes at least one of the 20 items
data_invoice_20 = data.loc[indices_list_invoice]
data_invoice_20.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [23]:
data_invoice_20.shape

(724141, 8)

In [24]:
data_invoice_20['StockCode'].value_counts()

85123A    5483
85099B    3847
21212     2874
84879     2779
21232     2380
          ... 
44228        1
20964        1
84933B       1
84559d       1
84967B       1
Name: StockCode, Length: 4812, dtype: int64

In [25]:
print('The Number of Columns needed For the Dataset')
print(len(data_invoice_20['StockCode'].unique()))

The Number of Columns needed For the Dataset
4812


In [26]:
print('The Number of Rows (invoices) needed For the Dataset')
print(len(data_invoice_20['InvoiceNo'].unique()))

The Number of Rows (invoices) needed For the Dataset
17881


In [27]:
data_summary = data_invoice_20.groupby('InvoiceNo')['StockCode'].apply(list)
data_summary

InvoiceNo
489434    [85048, 79323P, 79323W, 22041, 21232, 22064, 2...
489436    [48173C, 21755, 21754, 84879, 22119, 22142, 22...
489442    [21955, 22111, 22296, 84899E, 22271, 22272, 85...
489446    [21733, 85123A, 22178, 22125, 21671, 22086, 22...
489460    [79323P, 21977, 84991, 22274, 22273, 22179, 22...
                                ...                        
581497    [20719, 20723, 20724, 20727, 21212, 21238, 212...
581498    [15056bl, 20669, 20679, 20712, 20713, 20717, 2...
581538    [23193, 23194, 23084, 22068, 22956, 20936, 230...
581579    [22386, 22411, 21929, 85099C, 20713, 21931, 23...
581585    [22481, 22915, 22178, 22460, 84832, 23084, 848...
Name: StockCode, Length: 17881, dtype: object

### Question 2.1: Consolidate the items into 1 transaction per row and each product one-hot encoded.

In [28]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(data_summary).transform(data_summary)
final = pd.DataFrame(te_ary, columns=te.columns_)
final.head()

,10002,10002R,10080,10109,10120,10123C,10123G,10124A,10124G,10125,...,POST,SP1002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_70,gift_0001_80,m
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
final.shape

(17881, 4812)

We can see that 4812 is the number of items that are related to an invoice that includes at least one of the 20 most popular items.

### Question 2.2: Convert all the values to 1 when values are greater than 0 and 0 when values are 0 or less.

In [30]:
final_int = final*1
final_int.tail()

,10002,10002R,10080,10109,10120,10123C,10123G,10124A,10124G,10125,...,POST,SP1002,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_70,gift_0001_80,m
17876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Question 3.1: Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

In [31]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(final_int, min_support=0.07, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.084615,(20724)
1,0.122253,(20725)
2,0.072927,(20726)
3,0.095688,(20727)
4,0.081539,(20728)
5,0.078128,(20914)
6,0.083496,(21080)
7,0.151558,(21212)
8,0.076282,(21213)
9,0.073094,(21231)


### Question 3.2: Generate the association rules with their corresponding support, confidence and lift.

In [32]:
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6).iloc[:,:-2]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(21931),(85099B),0.109558,0.207874,0.071528,0.652884,3.140764
1,(22386),(85099B),0.118394,0.207874,0.075387,0.636750,3.063150


In [33]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(85099B),(21931),0.207874,0.109558,0.071528,0.344095,3.140764,0.048754,1.357578,1
1,(21931),(85099B),0.109558,0.207874,0.071528,0.652884,3.140764,0.048754,2.282021,1
2,(22386),(85099B),0.118394,0.207874,0.075387,0.636750,3.063150,0.050776,2.180663,1
3,(85099B),(22386),0.207874,0.118394,0.075387,0.362658,3.063150,0.050776,1.383255,1


### Question 4: Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.

| StockCode | Description |
| --- | --- |
| 21931 | JUMBO STORAGE BAG SUKI |
| 85099B | JUMBO BAG RED RETROSPOT |
| 22386 | JUMBO BAG PINK POLKADOT |

By looking at the options above it seems that a good rule would be to make a bundle or offer a discount on the `21931` | `JUMBO STORAGE BAG SUKI` when buying `85099B` | `JUMBO BAG RED RETROSPOT` which is one of 20 most popular items sold by the store.

### Question 5: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: 

I have no experience working with Frequent Itemset Mining, and for this particular assignment the bigger challenge was the data preparation to get the dataset ready to be used by the multiple libraries available at Python to find these Frequent items that could potentially lead to other items that might (or not) be frequent as well.

One of the main challenges during this assignment was getting the table group by the invoice, when that invoice included at least one of the popular items that we defined earlier (for this example we used the 20 most popular items). Initially I tried to do a dictionary where the key argument was the invoice, and the values were lists with the items that the person bought that particular time (these items must include at least of the items in the list of the 20 most popular items). After trying multiple times, I was able to find the `group_by` function that allowing me to do an even better/faster process.

I really see the application of this Algorithm (`Apriori` & `FP-Growth`) in multiple domains not only on the selection of related/frequent item and finding rules for this, but in every other situation where there are events that usually happens provided other event is happening too. For example, in my line of job working in the production of new airplanes, this analysis could help to determine some of the rules that could be happening in terms of the problems we see in the airplanes at different times. For example, if we have problems related to the wing structure if that usually is accompanied of issues with the fuel system or electromagnetic problems. By knowing this we could staff better certain areas with particular experts at different times. Of course, one must consider that probably there are better algorithms for this kind of problem, for as a first approach sounds good.